In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
#import tensorflow.compat.v1 as tf

In [5]:
!pwd

/home/neeraj/Documents/repos/tf1


In [6]:
embed = hub.Module("./sentence_wise_email/module/module_useT")

In [7]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

embed_fn = embed_useT('./sentence_wise_email/module/module_useT')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [8]:
import pandas as pd

In [10]:
mcat = pd.read_csv('MCAT Product and Service .xlsx - Sheet1.csv')

In [27]:
mcat

,MCAT_ID,MCAT_NAME,MCAT_TYPE
0,1,Adhesive Systems,P
1,2,Glue Applicators,P
2,3,Pressure Sensitive Adhesives,P
3,4,Agricultural Machinery,P
4,5,Drip Irrigation Systems,P
...,...,...,...
114317,235564,Starch Bag Making Machine,P
114318,235566,Quartz Single Distiller,P
114319,235567,SS Kitchen Jali,P
114320,235569,CPVC Pipe Scrap,P


In [14]:
adata=pd.read_csv("adata.csv")

In [16]:
pred=adata["prediction"].values

In [39]:
pred_1k=pred[0:10000]

In [11]:
mcat_list = mcat['MCAT_NAME'].values

In [12]:
%%time
mcat_matrix=embed_fn(mcat_list)

CPU times: user 8min 10s, sys: 1min 19s, total: 9min 30s
Wall time: 1min 18s


In [40]:
%%time
p1=[]
p2=[]
p3=[]
p4=[]
p5=[]
for i in range(10000):
    test_text = str(pred_1k[i])
    test_embed = embed_fn([test_text])
    sim_matrix  = np.inner(test_embed, mcat_matrix)
    sim_matrix=sim_matrix.reshape(len(mcat),)
    sim_matrix=np.array(sim_matrix)
    r=sim_matrix.argsort()[-5:][::-1]
    p1.append(mcat_list[r[0]])
    p2.append(mcat_list[r[1]])
    p3.append(mcat_list[r[2]])
    p4.append(mcat_list[r[3]])
    p5.append(mcat_list[r[4]])

CPU times: user 17min 24s, sys: 14min 49s, total: 32min 13s
Wall time: 4min 2s


In [41]:
adata_10k=adata.iloc[:10000,]

In [42]:
adata_10k["p1"]=p1
adata_10k["p2"]=p2
adata_10k["p3"]=p3
adata_10k["p4"]=p4
adata_10k["p5"]=p5

/home/neeraj/Documents/repos/tf1/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/neeraj/Documents/repos/tf1/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/neeraj/Documents/repos/tf1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [43]:
adata_10k["gt_mcat_lower"] = [s.lower() for s in adata_10k["gt_truth"]]

/home/neeraj/Documents/repos/tf1/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
adata_10k[adata_10k["prediction"]== adata_10k["gt_mcat_lower"]]

,user_search,prediction,gt_truth,p1,p2,p3,p4,p5,gt_mcat_lower
0,kirloskar electric motors 10 hp,kirloskar electric motors,Kirloskar Electric Motors,Kirloskar Electric Motors,Bonfiglioli Electric Motors,Havells Electric Motors,Kirloskar Industrial Motors,SPDP Motors,kirloskar electric motors
2,split ac fan motor,ac fan motor,AC Fan Motor,AC Fan Motor,Radiator Fan Motor,Electric Fan Motor,Exhaust Fan Motor,Cooler Fan Motor,ac fan motor
7,plastic bathroom bucket set,plastic buckets,Plastic Buckets,Plastic Buckets,Plastic Tubs,Plastic Jugs,Plastic Pumps,Plastic Bottles,plastic buckets
11,Shrink Tunnel,shrink tunnel,Shrink Tunnel,Shrink Tunnel,Hardening Tunnel,Inflatable Tunnel,Thin Wall Tube,Micro Tunnelling,shrink tunnel
12,fresh mutton near pune,fresh meat,Fresh Meat,Fresh Meat,Variety Meat,Ground Meat,Meat Snack,Silverside Meat,fresh meat
...,...,...,...,...,...,...,...,...,...
9984,Rubber Dumbbell Set,dumbbell set,Dumbbell Set,Dumbbell Set,Dumbbell Benches,Adjustable Dumbbell,Incline Dumbbell Bench,Dumbbells,dumbbell set
9987,Radium paint price,radium paint,Radium Paint,Radium Paint,Anti Radiation Paint,Graphite Paint,Metallic Spray Paint,Fluorescent Spray Paint,radium paint
9993,Nx audio dj control,dj equipment,DJ Equipment,DJ Equipment,DJ Accessories,DJ System,DJ Mixer,Pioneer DJ System,dj equipment
9997,distributors,pharmaceutical distributors,Pharmaceutical Distributors,Pharmaceutical Distributors,Pharmaceutical Third Party Manufacturing,Pharmaceutical Products,Pharmaceutical Third Party Manufacturing Service,Pharmaceutical Consultants,pharmaceutical distributors


In [46]:
(3905/10000)*100

39.050000000000004

In [47]:
adata_10k[(adata_10k["p1"]== adata_10k["gt_truth"]) | (adata_10k["p2"]== adata_10k["gt_truth"]) | (adata_10k["p3"]== adata_10k["gt_truth"]) | (adata_10k["p4"]== adata_10k["gt_truth"]) | (adata_10k["p5"]== adata_10k["gt_truth"]) ]

,user_search,prediction,gt_truth,p1,p2,p3,p4,p5,gt_mcat_lower
0,kirloskar electric motors 10 hp,kirloskar electric motors,Kirloskar Electric Motors,Kirloskar Electric Motors,Bonfiglioli Electric Motors,Havells Electric Motors,Kirloskar Industrial Motors,SPDP Motors,kirloskar electric motors
1,Skoda window regulator,window regulator,Window Regulators,Window Regulator Machine,Window Regulators,Ventilation Control Panel,Window Fittings,Ventilation Blowers,window regulators
2,split ac fan motor,ac fan motor,AC Fan Motor,AC Fan Motor,Radiator Fan Motor,Electric Fan Motor,Exhaust Fan Motor,Cooler Fan Motor,ac fan motor
5,dr james skin whitening cream,skin cream,Skin Creams,Anti-Itch Skin Cream,Skin Creams,Body Cream,Skin Cleansing Cream,Oily Skin Cream,skin creams
7,plastic bathroom bucket set,plastic buckets,Plastic Buckets,Plastic Buckets,Plastic Tubs,Plastic Jugs,Plastic Pumps,Plastic Bottles,plastic buckets
...,...,...,...,...,...,...,...,...,...
9989,Matt,matts,Rehab Matts,Rehab Matts,Cyclohexanone,Imarti,Pozzolan,Matt Paper,rehab matts
9993,Nx audio dj control,dj equipment,DJ Equipment,DJ Equipment,DJ Accessories,DJ System,DJ Mixer,Pioneer DJ System,dj equipment
9994,REVO bag closer DA-R,revo bag closer machine,Portable Bag Closer Machine,Revo Bag Closing Machine,Portable Bag Closer Machine,Bag Closer Machine,Bag Closer Machine Components,Slider Zip Bag,portable bag closer machine
9997,distributors,pharmaceutical distributors,Pharmaceutical Distributors,Pharmaceutical Distributors,Pharmaceutical Third Party Manufacturing,Pharmaceutical Products,Pharmaceutical Third Party Manufacturing Service,Pharmaceutical Consultants,pharmaceutical distributors


In [48]:
(5031/10000)*100

50.31

In [50]:
adata_10k[(adata_10k["p1"]== adata_10k["gt_truth"]) | (adata_10k["p2"]== adata_10k["gt_truth"])].shape

(4661, 9)

In [52]:
adata_10k[(adata_10k["p1"]== adata_10k["gt_truth"]) | (adata_10k["p2"]== adata_10k["gt_truth"]) | (adata_10k["p3"]== adata_10k["gt_truth"])].shape

(4841, 9)

In [51]:
(4661/10000)*100

46.61

In [6]:
pred=pd.read_csv("error.csv")

In [7]:
pred

,Unnamed: 0,user_search,prediction,gt_truth,dt_truth
0,3,it 1048st,it solut,st connector,st connector
1,6,Paver Poker Vibrator,concrete pack,needle vibr,needle vibrator
2,8,satake,sports satch,grain color sort,grain color sorter
3,9,Flameless Lighters,flameless light,electronic gas light,electronic gas lighters
4,10,Pratik Heat Products Pvt. Ltd,heat resign,herbal product,herbal products
...,...,...,...,...,...
57657,99993,orbit,orbit machin,orbital sand,orbital sander
57658,99995,manau packaging scrub machine,scrubbing machin,packaging machin,packaging machine
57659,99996,makka solam,maiz,maize grit,maize grits
57660,99997,Sliding mosquito net door,sliding mosquito net door,sliding mosquito net,sliding mosquito net


In [25]:
pred.iloc[1000:1100,]

,Unnamed: 0,user_search,prediction,gt_truth,dt_truth
1000,1713,fish medicine manufacturer,fish fashion medicin,fish breeding medicin,fish breeding medicine
1001,1716,Storeo,storeo management servic,blue star split air condition,blue star split air conditioners
1002,1717,Spoons made by wood machine manufacturer,wood machin,rolling pin,rolling pins
1003,1718,Nano fat harvest system,fat fater harvest system,medical equip,medical equipment
1004,1719,Electronic Organiser,electronic organ,pouch bag,pouch bags
...,...,...,...,...,...
1095,1860,oats powder,powder coating powd,oat flour,oat flour
1096,1862,Maestro silencer,motor silenc,silencer part,silencer parts
1097,1863,Oracle Scm Training,single train,oracle train,oracle training
1098,1864,mahendra baloro,mahindra car,round straw bal,round straw baler


In [52]:
gt_truth=pred["dt_truth"].values

In [53]:
%%time
gt_truth=gt_truth[1000:2000]

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 16.5 µs


In [54]:
query=pred["user_search"].values
query=query[1000:1100]


In [55]:
gt_truth

array(['fish breeding medicine', 'blue star split air conditioners',
       'rolling pins', 'medical equipment', 'pouch bags',
       'loop handle bag', 'solar power pack', 'gypsum plaster',
       'cnc turning machine', 'plastic injection moulding machine',
       'advertising displays', 'eco-friendly bags', 'office diary',
       'candy lollipop', 'ph buffer solution', 'decorative items',
       'handmade earring', 'industrial mixers', 'double wheel barrow',
       'juttis', 'konica flex printing machine', 'lip cosmetics',
       'mens casual blazer', 'raw cotton', 'ayurvedic tablets',
       'mens underwear', 'lithium polymer battery', 'printing services',
       'turning machine', 'motor starters', 'cat wheel loader',
       'para nitro toluene ortho sulphonic acid', 'floor safety signs',
       'hard anodized roti tawa', 'juki industrial sewing machine',
       'mobile phones', 'plastic pipe making machine', 'plywoods',
       'water tanks', 'file folders', 'police cap',
       'n

In [33]:
query

array(['fish medicine manufacturer', 'Storeo',
       'Spoons made by wood machine manufacturer',
       'Nano fat harvest system', 'Electronic Organiser', 'Handle bags',
       'solar batteries 100ah 12 volt', 'gypsum bricks price',
       'durga machinery', 'plastic molding injection chair machine',
       'Lg h 440n mobile display', 'printed eco friendly bags',
       'mens office bags small size', 'lollipop candy', 'ph buffers',
       'seagull', 'handicraft earrings',
       'heavy duty mixer machine for industrial use', 'in lucknow',
       'Maharaja shoes', 'MONOTECH', 'lipice sheer color natural 2g',
       'jodhpuri chokt', 'kapas', 'fizz tablet', 'menstural underwear',
       'li polymer battery', 'Rayon fabric dark print', 'dx 160',
       'paragon starter', 'gua catting mashine',
       'para-toluene sulphonic acid in jumbo bag', 'Floor Safety Signs',
       'hard anodised tawa',
       'Juki industrial sewing machine price in kenya', 'SELFIIE',
       'electric pipe making

In [20]:
pred_list

array(['fish fashion medicin', 'storeo management servic', 'wood machin',
       'fat fater harvest system', 'electronic organ', 'handle bag',
       'solar batteri', 'gypsum brick', 'durga standing machin',
       'plastic chair shirt chair shir'], dtype=object)

In [56]:
%%time
mcat_matrix=embed_fn(gt_truth)

CPU times: user 8.92 s, sys: 400 ms, total: 9.32 s
Wall time: 1.34 s


In [97]:
test_text = "helmet for two wheeler"
test_embed = embed_fn([test_text])

In [98]:
sim_matrix  = np.inner(test_embed, mcat_matrix)
sim_matrix

array([[ 0.21076399,  0.28799006,  0.35958257,  0.40186065,  0.39201456,
         0.4983987 ,  0.32961237,  0.20985678,  0.36021757,  0.34196365,
         0.266764  ,  0.3499121 ,  0.06356171,  0.17861524,  0.09020568,
         0.30310827,  0.30217588,  0.17769548,  0.52341187,  0.24160871,
         0.20362063,  0.20394081,  0.30372167,  0.1765806 ,  0.1239911 ,
         0.31201547,  0.34067482,  0.10181478,  0.35205263,  0.44129005,
         0.4358334 ,  0.17763564,  0.31994262,  0.22361845,  0.35970664,
         0.16185732,  0.3103338 ,  0.2315666 ,  0.3458551 ,  0.12982973,
         0.45865443,  0.2674261 ,  0.21321833,  0.21840203,  0.27882928,
         0.350824  ,  0.26540637,  0.04478219,  0.2952178 ,  0.24026328,
        -0.00354399,  0.4050337 ,  0.15137696,  0.30352587,  0.34883267,
         0.09239504,  0.23199116,  0.2789151 ,  0.10476007,  0.1449182 ,
         0.12277661,  0.29528672,  0.35196775,  0.13181433,  0.14058727,
         0.16821627,  0.38781178,  0.38221467,  0.1

In [99]:
sim_matrix=sim_matrix.reshape(1000,)

In [123]:
sim_matrix=np.array(sim_matrix)
r=sim_matrix.argsort()[-3:][::-1]


In [124]:
r

array([ 89, 119, 399])

In [126]:
for i in range(3):
    print(gt_truth[r[i]])

motorcycle helmets
headlight visor
bike accessories


In [101]:
gt_truth[89]

'motorcycle helmets'

In [102]:
gt_truth[119]

'headlight visor'

In [103]:
gt_truth[399]

'bike accessories'

In [22]:
pred_matrix = embed_fn(pred_list)

In [104]:
pred[pred["user_search"]== "helmet for two wheeler"]

,Unnamed: 0,user_search,prediction,gt_truth,dt_truth
1089,1854,helmet for two wheeler,two wheeler helmet,motorcycle helmet,motorcycle helmets


In [143]:
sample_data=pred.iloc[0:10000:,]

In [144]:
sample_data=sample_data[["user_search","prediction","dt_truth"]]

In [145]:
sample_data=sample_data.rename(columns={'dt_truth':"gt_truth"})

In [146]:
gt_truth=sample_data['gt_truth'].values

In [147]:
query=sample_data['user_search'].values

In [148]:
query.shape

(10000,)

In [149]:
%%time
mcat_matrix=embed_fn(gt_truth)

CPU times: user 1min 17s, sys: 9.66 s, total: 1min 26s
Wall time: 11.8 s


In [151]:
%%time
p1=[]
p2=[]
p3=[]
for i in range(10000):
    test_text = str(query[i])
    test_embed = embed_fn([test_text])
    sim_matrix  = np.inner(test_embed, mcat_matrix)
    sim_matrix=sim_matrix.reshape(10000,)
    sim_matrix=np.array(sim_matrix)
    r=sim_matrix.argsort()[-3:][::-1]
    p1.append(gt_truth[r[0]])
    p2.append(gt_truth[r[1]])
    p3.append(gt_truth[r[2]])


CPU times: user 9min 18s, sys: 10min 9s, total: 19min 28s
Wall time: 2min 29s


In [152]:
sample_data['p1']=p1
sample_data['p2']=p2
sample_data['p3']=p3

In [133]:
sample_data[sample_data["p1"]==sample_data['gt_truth']]

,user_search,prediction,gt_truth,p1,p2,p3
1000,fish medicine manufacturer,fish fashion medicin,fish breeding medicine,fish breeding medicine,food processing consultants,speciality chemicals
1007,gypsum bricks price,gypsum brick,gypsum plaster,gypsum plaster,ppc cement,granite slabs
1009,plastic molding injection chair machine,plastic chair shirt chair shir,plastic injection moulding machine,plastic injection moulding machine,automatic rigid box making machine,juki industrial sewing machine
1011,printed eco friendly bags,eco friendly bag,eco-friendly bags,eco-friendly bags,canvas bags,bag packaging machines
1013,lollipop candy,candy jelli,candy lollipop,candy lollipop,pistachio,chocolate paste
...,...,...,...,...,...,...
1984,Fire Extinguisher Refilling in Kanakapura Road...,residential estate servic,fire extinguisher refilling,fire extinguisher refilling,safex fire extinguishers,hykon solar water heater
1987,glass hookah,hookah,glass hookah,glass hookah,stainless steel rectangle pipe,hookah accessories
1990,makka rat,maize se,rat repellent,rat repellent,gpon olt,paecilomyces lilacinus
1998,dol auto switch,automatic switch,motor automatic switch,motor automatic switch,reverse switch,ignition switches


In [134]:
sample_data[sample_data["prediction"]==sample_data['gt_truth']]

,user_search,prediction,gt_truth,p1,p2,p3


In [135]:
sample_data[sample_data["p2"]==sample_data['gt_truth']]

,user_search,prediction,gt_truth,p1,p2,p3
1034,Juki industrial sewing machine price in kenya,sewing machin,juki industrial sewing machine,juki industrial sewing machine,juki industrial sewing machine,non woven bag making machine
1043,sprite drink,strumb,cold drink,cold drink,cold drink,caustic soda
1044,Metal external staircase,external staircas,staircase,stainless steel staircase railing,staircase,prefabricated cabin
1057,qsc rmx amplifiers,amplifi,power amplifiers,akg conduit pipes,power amplifiers,subwoofer speaker
1058,potato waffers,potato osmosis waf,potato wafers,potato chips,potato wafers,seed potatoes
...,...,...,...,...,...,...
1851,lithium ion battery for solar storage,lithium batteri,lithium ion battery,lithium polymer battery,lithium ion battery,solar batteries
1856,Scrap container,steel contain,shipping containers,containers lid,shipping containers,plastic containers
1877,soy chunks,soy chunk,soya chunk,soya chunk,soya chunk,whey protein
1911,Robot Shield,robotic shield,arduino shield,beam load cell,arduino shield,robot spare parts


In [137]:
sample_data[sample_data["p3"]==sample_data['gt_truth']].shape

(43, 6)

In [139]:
sample_data.to_csv("universal_sentence_encoder_result.csv",index=False)

In [153]:
sample_data[(sample_data["p1"]==sample_data['gt_truth']) | (sample_data["p2"]==sample_data['gt_truth']) | (sample_data["p3"]==sample_data['gt_truth']) ].shape

(3052, 6)

In [154]:
sample_data[sample_data["prediction"]==sample_data["gt_truth"]].shape

(0, 6)

In [162]:
df=pd.read_csv("Correction  spreadsheet - Sheet1.csv")

In [163]:
df.columns

Index(['user_search', 'GT_MCAT', 'prediction', 'result', 'ground truth e',
       'jumbled'],
      dtype='object')

In [164]:
df=df[["user_search","prediction","GT_MCAT"]]

In [165]:
gt_truth=df['GT_MCAT'].values
query=df['user_search'].values


In [166]:
%%time
mcat_matrix=embed_fn(gt_truth)

CPU times: user 2min 24s, sys: 17.2 s, total: 2min 41s
Wall time: 22 s


In [167]:
%%time
p1=[]
p2=[]
p3=[]
for i in range(20000):
    test_text = str(query[i])
    test_embed = embed_fn([test_text])
    sim_matrix  = np.inner(test_embed, mcat_matrix)
    sim_matrix=sim_matrix.reshape(20000,)
    sim_matrix=np.array(sim_matrix)
    r=sim_matrix.argsort()[-3:][::-1]
    p1.append(gt_truth[r[0]])
    p2.append(gt_truth[r[1]])
    p3.append(gt_truth[r[2]])

CPU times: user 21min 17s, sys: 21min 22s, total: 42min 39s
Wall time: 5min 39s


In [172]:
df

,user_search,prediction,GT_MCAT,p1,p2,p3
0,mens sports t shirt,mens sports t shirt,Mens Sports T-Shirt,Mens Sports T-Shirt,Mens Polo T Shirt,Mens Polo T Shirt
1,mens hooded sweatshirt,mens hooded sweatshirts,Man Hoodies,Mens Hooded T Shirt,Mens Denim Jacket,Mens Denim Shirt
2,Candle Stand,candle stands,Candle Stands,Candle Stands,Candle Stands,Candle Stands
3,polthin pecking bags colars,polyter bags,Polythene Bags,Potli Bags,Potli Bags,Potli Bags
4,nt cutter d-1000,cutting machine,Cable Cutters,End Cutter,Cross Hatch Cutter Elcometer,Stone Jali
...,...,...,...,...,...,...
19995,solid state relay jaipur,solid state relay,Solid State Relay,Bharat Alternator,Mysore pak,Bajaj Geyser
19996,Fander laine,laboratory chemical,Gas Boiler,Kalamkari Kurtis,Chikan Kurtis,Tazret Forte
19997,rice mill at chennai,rice mill,Rice Mill,Rice Mill,Rice Mill,Rice Mill
19998,KADAI 10INCH 12 INCH SS,stainless steel kadai,SS Glass Bracket,Duro Plywood,Nylon 66 Granule,Century Laminates


In [168]:
df['p1']=p1
df['p2']=p2
df['p3']=p3

In [173]:
df[(df["p1"]==df['GT_MCAT']) | (df["p2"]==df['GT_MCAT']) | (df["p3"]==df['GT_MCAT']) ]

,user_search,prediction,GT_MCAT,p1,p2,p3
0,mens sports t shirt,mens sports t shirt,Mens Sports T-Shirt,Mens Sports T-Shirt,Mens Polo T Shirt,Mens Polo T Shirt
2,Candle Stand,candle stands,Candle Stands,Candle Stands,Candle Stands,Candle Stands
7,teak wood beading,wooden beading,Wooden Beading,Wooden Beading,Teak Wood,Teak Wood
9,barcode printers,barcode printers,Barcode Printers,Barcode Printers,Barcode Printers,Barcode Printers
14,gold plated connectors,gold plated connector,Gold Connector,Gold Connector,Copper Nickel Fasteners,Copper Flexible Cable
...,...,...,...,...,...,...
19988,Plastic Lunch Box,plastic lunch box,Plastic Lunch Box,Plastic Lunch Box,Plastic Food Bag,Cardboard Tray
19990,amla shikakai shampoo,amla shikakai shampoo,Amla Shikakai Shampoo,Amla Shikakai Shampoo,Dabur Amla Hair Oil,Herbal Shampoo
19993,Designer mangalore tiles,designer tiles,Designer Tiles,Designer Tiles,Designer Tiles,Designer Tiles
19997,rice mill at chennai,rice mill,Rice Mill,Rice Mill,Rice Mill,Rice Mill


In [176]:
df["gt_mcat"] = [s.lower() for s in df["GT_MCAT"]]

In [177]:
df[df["prediction"]==df["gt_mcat"]].shape

(7301, 7)

In [178]:
7301/20000

0.36505

In [179]:
9050/20000

0.4525

In [180]:
df[df["p1"]==df["GT_MCAT"]].shape

(7570, 7)

In [181]:
7570/20000

0.3785